# **Import necessary libraries**

In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import csv 
import psycopg2  # Enables connection and interaction with PostgreSQL databases
import logging

In [33]:
# Specify the path to the CSV file containing data
df=pd.read_csv('/Users/sebastiangurgel/Desktop/Masteryproject1/query_results-2023-08-22_85505.csv', low_memory=False)

In [34]:
# checking the Dataset for columns
df.head()

,session_id,user_id,trip_id,session_start,session_end,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation,user_id_1,birthdate,age,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,trip_id_1,origin_airport,destination,destination_airport,seats,return_flight_booked,departure_time,return_time,checked_bags,trip_airline,destination_airport_lat,destination_airport_lon,haversine_distance,base_fare_usd,trip_id_2,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd
0,101486-c431d39dbe884b6f9d6a267fe6655e94,101486,101486-1015905607d74b15954bfd4ac7029ef3,2023-06-01 09:00:00,2023-06-01 09:02:38,False,False,NaN,NaN,True,True,21,False,101486,1972-12-07,50,F,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17,101486-1015905607d74b15954bfd4ac7029ef3,TCM,edmonton,YED,1.0,True,2023-06-10 10:00:00,2023-06-14 10:00:00,0.0,United Airlines,53.667,-113.467,995.681600,189.91,101486-1015905607d74b15954bfd4ac7029ef3,Crowne Plaza - edmonton,3.0,1.0,2023-06-10 13:12:24.03,2023-06-14 11:00:00,253.0
1,101486-c668e4e44ffc4e5a93c46f661320aa23,101486,101486-6759c5dd49a1457d916bb2bbf48c3115,2023-06-17 19:42:00,2023-06-17 19:44:37,False,False,NaN,NaN,False,True,21,False,101486,1972-12-07,50,F,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101486-6759c5dd49a1457d916bb2bbf48c3115,Banyan Tree - montreal,5.0,2.0,2023-06-24 11:00:00,2023-06-29 11:00:00,144.0
2,101961-c2ff557a18294bff9856f4d14ced1a47,101961,101961-19f641633ebc442799662326f0b4dfa0,2023-04-24 19:28:00,2023-04-24 19:30:19,True,False,0.1,NaN,True,True,19,False,101961,1980-09-14,42,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17,101961-19f641633ebc442799662326f0b4dfa0,BOS,new york,LGA,1.0,True,2023-05-05 11:00:00,2023-05-09 11:00:00,0.0,Allegiant Air,40.640,-73.779,297.807805,49.67,101961-19f641633ebc442799662326f0b4dfa0,Conrad - new york,3.0,1.0,2023-05-05 13:22:30.72,2023-05-09 11:00:00,165.0
3,101961-83fd9718a030497ca70b55697ecba1b7,101961,101961-29a8ff7c9910469c959fffa60215cf78,2023-02-02 12:39:00,2023-02-02 12:41:23,False,False,NaN,NaN,True,True,19,False,101961,1980-09-14,42,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17,101961-29a8ff7c9910469c959fffa60215cf78,BOS,montreal,YHU,1.0,True,2023-02-08 07:00:00,2023-02-13 07:00:00,1.0,United Airlines,45.517,-73.417,402.714696,77.02,101961-29a8ff7c9910469c959fffa60215cf78,Rosewood - montreal,4.0,1.0,2023-02-08 09:30:00.99,2023-02-12 11:00:00,197.0
4,101961-2bc80c62f0284004bd93b2674c69748e,101961,101961-836fd88487d240baa4402c8e4c6f188c,2023-03-12 17:56:00,2023-03-12 17:59:06,False,False,NaN,NaN,True,True,25,False,101961,1980-09-14,42,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17,101961-836fd88487d240baa4402c8e4c6f188c,BOS,seattle,BFI,1.0,True,2023-03-16 07:00:00,2023-03-21 07:00:00,1.0,Kenmore Air,47.530,-122.302,4253.525872,769.50,101961-836fd88487d240baa4402c8e4c6f188c,Extended Stay - seattle,4.0,1.0,2023-03-16 14:00:14.67,2023-03-21 11:00:00,132.0


# *Preparing the CSV for insertation in a SQL-Database*

this code is about reading data from a CSV file transforming that data, and inserting it into a PostgreSQL database. It starts by connecting to the database, reading the CSV file, and then for each row in the CSV, it processes the data, prepares it for insertion, and finally inserts it into the database. After processing all the rows, it saves the changes and closes the connection

In [38]:
import csv
import psycopg2

# PostgreSQL connection parameters
db_params = {
    'hostname': 'localhost',
    'port': '5432',
    'database': 'travel_tide_aggregations',
    'username': 'postgres',
    'password': 'XXXX'
}

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=db_params['hostname'],
    port=db_params['port'],
    dbname=db_params['database'],
    user=db_params['username'],
    password=db_params['password']
)

# Open a cursor to perform database operations
cursor = conn.cursor()

# Define the path to your CSV file
csv_file_path ='/Users/sebastiangurgel/Desktop/Masteryproject1/query_results-2023-08-22_85505.csv'

# Open and read the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)  # Read CSV data as dictionaries
    
     # Loop through each row in the CSV file
    for row in csv_reader:  
  
      
       
        # Check if specific columns are empty, and if so, set them to None or default values
        
        departure_time = row['departure_time']
        if departure_time == '':
            departure_time = None

        return_time = row['return_time']
        if return_time == '':
            return_time = None

        check_in_time = row['check_in_time']
        if check_in_time == '':
            check_in_time = None

        check_out_time = row['check_out_time']
        if check_out_time == '':
            check_out_time = None
     
        seats = float(row['seats']) if row['seats'] != '' else 0
        flight_discount_amount = float(row['flight_discount_amount']) if row['flight_discount_amount'] != '' and not pd.isna(row['flight_discount_amount']) else 0
        hotel_discount_amount = float(row['hotel_discount_amount']) if row['hotel_discount_amount'] != '' and not pd.isna(row['hotel_discount_amount']) else 0
        checked_bags = float(row['checked_bags']) if row['checked_bags'] != '' and not pd.isna(row['checked_bags']) else 0
        destination_airport_lat = float(row['destination_airport_lat']) if row['destination_airport_lat'] != '' and not pd.isna(row['destination_airport_lat']) else 0
        destination_airport_lon = float(row['destination_airport_lon']) if row['destination_airport_lon'] != '' and not pd.isna(row['destination_airport_lon']) else 0
        base_fare_usd = float(row['base_fare_usd']) if row['base_fare_usd'] != '' and not pd.isna(row['base_fare_usd']) else 0
        nights = float(row.get('nights', 0)) if row.get('nights', '') != '' and not pd.isna(row.get('nights', '')) else 0
        rooms = float(row['rooms']) if row['rooms'] != '' and not pd.isna(row['rooms']) else 0
        hotel_per_room_usd = float(row['hotel_per_room_usd']) if row['hotel_per_room_usd'] != '' and not pd.isna(row['hotel_per_room_usd']) else 0
        haversine_distance = float(row['haversine_distance']) if row['haversine_distance'] != '' and not pd.isna(row['haversine_distance']) else 0
        age = int(row['age'])
        
        
        # Map data from CSV columns to corresponding database columns
        # Each column is assigned a value from the CSV row
        row_data = {
            'user_id': int(row['user_id']),
            'birthdate': row['birthdate'],
            'gender': row['gender'],
            'married': bool(row['married'].lower() == 'true'),
            'has_children': bool(row['has_children'].lower() == 'true'),
            'home_country': row['home_country'],
            'home_city': row['home_city'],
            'home_airport': row['home_airport'],
            'home_airport_lat': float(row['home_airport_lat']),
            'home_airport_lon': float(row['home_airport_lon']),
            'sign_up_date': row['sign_up_date'],
            'user_id_1': int(row['user_id_1']),
            'session_id': row['session_id'],
            'trip_id': row['trip_id'],
            'session_start': row['session_start'],
            'session_end': row['session_end'],
            'flight_discount': bool(row['flight_discount']),
            'hotel_discount': bool(row['hotel_discount'].lower() == 'true'),
            'flight_discount_amount': flight_discount_amount,
            'hotel_discount_amount': hotel_discount_amount,
            'flight_booked': bool(row['flight_booked'].lower() == 'true'),
            'hotel_booked': bool(row['hotel_booked'].lower() == 'true'),
            'page_clicks': float(row['page_clicks']),
            'cancellation': bool(row['cancellation'].lower() == 'true'),
            'trip_id_1': row['trip_id_1'],
            'origin_airport': row['origin_airport'],
            'destination': row['destination'],
            'destination_airport': row['destination_airport'],  
            'seats': seats,
            'return_flight_booked': row['return_flight_booked'],
            'departure_time': departure_time,
            'return_time': return_time,
            'checked_bags': checked_bags,
            'trip_airline': row['trip_airline'],
            'destination_airport_lat': destination_airport_lat,
            'destination_airport_lon': destination_airport_lon,
            'base_fare_usd': base_fare_usd,
            'trip_id_2': row['trip_id_2'],
        
            'hotel_name': row['hotel_name'],
            'nights': nights, 
            'rooms': rooms,
            'check_in_time': check_in_time,
            'check_out_time': check_out_time,
            'hotel_per_room_usd': hotel_per_room_usd,
            'haversine_distance': haversine_distance,
            'age': age
        }
        
        # Define the SQL query for inserting data into the database
        # List of column names corresponding to the database table
        # Values will be inserted into these columns
        insert_query = """
            INSERT INTO travel_tide_aggregations (
            
                user_id, birthdate, gender, married, has_children, home_country,
                home_city, home_airport, home_airport_lat, home_airport_lon,
                sign_up_date, user_id_1, session_id, trip_id,
                session_start, session_end, flight_discount, hotel_discount,
                flight_discount_amount, hotel_discount_amount, flight_booked,
                hotel_booked, nights, page_clicks, cancellation, trip_id_1,
                origin_airport, destination, destination_airport, seats,
                return_flight_booked, departure_time, return_time,
                checked_bags, trip_airline, destination_airport_lat,
                destination_airport_lon, base_fare_usd, trip_id_2,
                hotel_name, rooms, check_in_time, check_out_time, 
                hotel_per_room_usd, haversine_distance, age 
            ) VALUES (
                %(user_id)s, %(birthdate)s, %(gender)s, %(married)s, %(has_children)s,
                %(home_country)s, %(home_city)s, %(home_airport)s,
                %(home_airport_lat)s, %(home_airport_lon)s, %(sign_up_date)s,
                %(user_id_1)s, %(session_id)s, %(trip_id)s,
                %(session_start)s, %(session_end)s, %(flight_discount)s,
                %(hotel_discount)s, %(flight_discount_amount)s,
                %(hotel_discount_amount)s, %(flight_booked)s, %(hotel_booked)s, %(nights)s,
                %(page_clicks)s, %(cancellation)s, %(trip_id_1)s,
                %(origin_airport)s, %(destination)s, %(destination_airport)s,
                %(seats)s, %(return_flight_booked)s, %(departure_time)s,
                %(return_time)s, %(checked_bags)s, %(trip_airline)s,
                %(destination_airport_lat)s, %(destination_airport_lon)s,
                %(base_fare_usd)s, %(trip_id_2)s, %(hotel_name)s, %(rooms)s,
                %(check_in_time)s, %(check_out_time)s, %(hotel_per_room_usd)s,
                 %(haversine_distance)s, %(age)s
            );
        """
        
        
        cursor.execute(insert_query, row_data)  # Execute the SQL query with the row's data


    # Commit changes and close the connection
    conn.commit()  # Apply the changes
    conn.close()  # Close the connection
